# Data transformation of ml_latest_small data set

#### TODO:
1. How many movies are in data set ?

2. What is the most common genre of movie?

3. What are top 10 movies with highest rate ?

4. What are 5 most often rating users ?

5. When was done first and last rate included in data set and what was the rated movie tittle?

6. Find all movies released in 1990

Importing libraries

In [12]:
import time
from pandas import DataFrame
from sqlalchemy import create_engine
import pandas as pd

Setting up connection to db

In [14]:
# Connect to the database
db_name = 'database'
db_user = 'username'
db_pass = 'secret'
db_host = 'db'
db_port = '5432'

db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

#### 1. How many movies is present within dataset 'ml_latest_small' ?

In [10]:
def find_movies_nr():
    """
    Aim of this method is to find how many movies is present within dataset 'ml_latest_small'.
    As per readme.txt file "Each line of this file (movies.csv) after the header row represents one movie"
    :return number of movies in dataset, distinct number of titles, repeated titles if exist
    """
    # How many movies are in data set ?
    query_titles= "SELECT count(*) as cnt "\
                  "FROM movies"
    nr_movies = pd.read_sql_query(query_titles, db)

    # How many distinct titles ?
    query_titles= "SELECT count(distinct m.title) as cnt " \
                  "FROM movies m"
    nr_titles = pd.read_sql_query(query_titles, db)

    # How many distinct movie IDs ?
    query_movie_ids= "SELECT count(distinct m.movieId) as cnt "\
                     "FROM movies m"
    nr_movie_ids = pd.read_sql_query(query_movie_ids, db)

    # Check if dataset incorporates any repeated movie titles. If so, which of them are repeated ? 
    # As per readme.txt : "Errors and inconsistencies may exist in these titles." 
    repeated_titles_list = None
    if nr_movie_ids['cnt'][0] > nr_titles['cnt'][0]:
        query_repeated_titles= "SELECT m.title as title "\
                               "FROM movies m "\
                               "GROUP BY m.title "\
                               "HAVING COUNT(m.title) > 1 ;"
        df_repeated_titles = pd.read_sql_query(query_repeated_titles, db)
        repeated_titles_list = df_repeated_titles['title'].tolist()

    return nr_movies['cnt'][0], nr_titles['cnt'][0] , repeated_titles_list

#### 1. Result:

In [11]:
print("Dataset has {} movies and {} distinct titles. Repeated movie titles are " \
              "{}".format(*find_movies_nr()))

Dataset has 9742 movies and 9737 distinct titles. Repeated movie titles are ['Eros (2004)', 'Saturn 3 (1980)', 'Confessions of a Dangerous Mind (2002)', 'Emma (1996)', 'War of the Worlds (2005)']


#### 2. What is the most common genre of movie?

In [15]:
def most_common_genre():
    """
    Aim of this method is to find out what is the most common genre of movie?
    :return most common genre of movie (string)
    """

    # Fetch movies table to pandas DataFrame
    df_movies = pd.read_sql_query("SELECT * "\
                                  "FROM movies ;", db)
    df_movies.rename(columns=df_movies.iloc[0])

    # Split genres column given pipe seperator
    split_genres = df_movies.genres.str.split('|')
    splitted_df = pd.DataFrame({'genres':split_genres.sum(),'title':df_movies.title.repeat(split_genres.str.len())})

    # Calculate which genre is the most common one
    common_genre = splitted_df['genres'].value_counts().idxmax()
    common_genre_nr = splitted_df['genres'].value_counts()[0]

    return common_genre, common_genre_nr

#### 2. Result:

In [16]:
print("The most common genre is {}. It appears {} times.".format(*most_common_genre()))

The most common genre is Drama. It appears 4361 times.


#### 3. What are top 10 movies with highest rate ?

In [7]:
def top_rated_movies():
    """
    Aim of this method is to verify what are top 10 movies with highest rate ?
    :return pandas DataFrame with top 10 rated movies
    """

    query = "SELECT movies.movieid, title, count " \
            "FROM movies "\
            "JOIN (SELECT movieid, count(*) AS count "\
            "FROM ratings "\
            "WHERE rating = 5 "\
            "GROUP BY movieid "\
            "ORDER BY count "\
            "DESC LIMIT 10) t " \
            "ON movies.movieid = t.movieid ORDER BY 3 desc ; "

    # Fetch data from sql filtered to top 10 records with highest rate
    top_movies_df = pd.read_sql_query(query, db)

    return top_movies_df

#### 3. Result:

In [17]:
print("Top 10 rated movies are: {}".format(top_rated_movies()))

Top 10 rated movies are:    movieid                                              title  count
0      318                   Shawshank Redemption, The (1994)    153
1      296                                Pulp Fiction (1994)    123
2      356                                Forrest Gump (1994)    116
3     2571                                 Matrix, The (1999)    109
4      260          Star Wars: Episode IV - A New Hope (1977)    104
5      593                   Silence of the Lambs, The (1991)     92
6      527                            Schindler's List (1993)     92
7      858                              Godfather, The (1972)     88
8     2959                                  Fight Club (1999)     81
9     1196  Star Wars: Episode V - The Empire Strikes Back...     80


#### 4. What are 5 most often rating users ?

In [8]:
def top_rated_users():
    """
    Aim of this method is to verify what are 5 most often rating users ?
    :return df of users that are most often rated together with number of made ratings
    """
    query = "SELECT r.userId, COUNT(r.userId) AS cnt "\
            "FROM ratings r "\
            "GROUP BY r.userId "\
            "ORDER BY cnt DESC "\
            "LIMIT 5;"
    
    # Fetch data from ratings table filtered as per requirement
    top_users_df = pd.read_sql_query(query, db)

    # Extract list of most often rating users
    top_users_list = top_users_df['userid'].tolist()

    return top_users_df

#### 4. Result:

In [19]:
print("Top 5 most often rating users: {}".format(top_rated_users()))

Top 5 most often rating users:    userid   cnt
0     414  2698
1     599  2478
2     474  2108
3     448  1864
4     274  1346


#### 5. When was done first and last rate included in data set and what was the rated movie tittle?

In [9]:
def run_first_and_last_rate():
    """
    Aim of this method is to find out when was done first and last rate included in data set
    and what was the rated movie title?
    :return first rate date, firstly rated movie title(s), last rate date, lastly rated movie title(s)
    """
    query = "SELECT r.timestamp as tmp, m.title, "\
            "'min' as level from ratings r "\
            "LEFT JOIN movies m on m.movieId=r.movieId "\
            "WHERE r.timestamp = (select min(r.timestamp) FROM ratings r) " \
            "UNION "\
            "SELECT r.timestamp as tmp, m.title, "\
            "'max' as level "\
            "FROM ratings r "\
            "LEFT JOIN movies m on m.movieId=r.movieId "\
            "WHERE r.timestamp = (select max(r.timestamp) from ratings r) ;"  

    # Fetch data from db including first and last rate date and corresponding movie title(s)
    ratings = pd.read_sql_query(query, db)

    # Limit DataFrame to have only rows related first rate date
    first_rated_df = ratings.loc[ratings['level'] == 'min']
    # Extract first available rate date
    first_rate_date = first_rated_df['tmp'][0]
    first_rate_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_rate_date))
    # Extract firstly rated movie title(s)
    first_rate_movies = first_rated_df['title'].tolist()

    # Limit DataFrame to have only rows related last rate date
    last_rated_df = ratings[ratings['level'] == 'max']
    # Extract last available rate date
    last_rate_date = last_rated_df['tmp'].tolist()[0]
    last_rate_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(last_rate_date))
    # Extract last rated movie title(s)
    last_rate_movies = last_rated_df['title'].tolist()

    return first_rate_date, first_rate_movies, last_rate_date, last_rate_movies

#### 5. Result:

In [20]:
print("First rating was made in {} and associated titles list that was found is {}, " \
              "Last rating was made in {} and associated titles list that was found is {}". \
              format(*run_first_and_last_rate()))

First rating was made in 1996-03-29 18:36:55 and associated titles list that was found is ['Aladdin (1992)', 'Apollo 13 (1995)', 'Baby-Sitters Club, The (1995)', 'Batman (1989)', 'Beauty and the Beast (1991)', 'Beverly Hills Cop III (1994)', 'Black Beauty (1994)', 'Boys on the Side (1995)', 'Circle of Friends (1995)', "City Slickers II: The Legend of Curly's Gold (1994)", 'Clear and Present Danger (1994)', 'Cliffhanger (1993)', 'Copycat (1995)', 'Corrina, Corrina (1994)', 'Crimson Tide (1995)', 'Dances with Wolves (1990)', 'Die Hard: With a Vengeance (1995)', 'Disclosure (1994)', 'Drop Zone (1994)', 'Englishman Who Went Up a Hill But Came Down a Mountain, The (1995)'], Last rating was made in 2018-09-24 14:27:30 and associated titles list that was found is ['Crumb (1994)']


#### 6. Find all movies released in 1990

In [10]:
def movies_per_release_date():
    """
    Aim of this method is to find all movies released in 1990
    :return number of movies released in 1990 together with corresponding movie titles
    """

    query = "SELECT title as title "\
            "FROM movies "\
            "WHERE title ~* '(1990)';"

    # Fetch movies from db limited to these which have '1990' number within its title
    movies_filtered_per_release_date = pd.read_sql_query(query, db)
    movies_list = movies_filtered_per_release_date['title'].tolist()

    # Filter movies titles that have 1990 year present in parentheses as PostGreSQL omits them
    matching_titles = [title for title in movies_list if "(1990)" in title]

    # Count how many movies were released in 1990
    cnt_movies = len(matching_titles)
    return cnt_movies, matching_titles

#### 6. Result:

In [21]:
print("Dataset incorporates of {} movies released in 1990. They are as follows: {}". \
      format(*movies_per_release_date()))

Dataset incorporates of 147 movies released in 1990. They are as follows: ['Home Alone (1990)', 'Ghost (1990)', 'Dances with Wolves (1990)', 'Pretty Woman (1990)', 'Days of Thunder (1990)', 'Grifters, The (1990)', 'Tie Me Up! Tie Me Down! (¡Átame!) (1990)', 'Paris Is Burning (1990)', 'Goodfellas (1990)', 'Trust (1990)', 'Rosencrantz and Guildenstern Are Dead (1990)', "Miller's Crossing (1990)", 'Femme Nikita, La (Nikita) (1990)', 'Pump Up the Volume (1990)', 'Cyrano de Bergerac (1990)', 'Amityville Curse, The (1990)', 'Die Hard 2 (1990)', 'Young Guns II (1990)', 'Marked for Death (1990)', 'Hunt for Red October, The (1990)', 'King of New York (1990)', 'Metropolitan (1990)', "Child's Play 2 (1990)", 'Exorcist III, The (1990)', 'Gremlins 2: The New Batch (1990)', 'Back to the Future Part III (1990)', 'Godfather: Part III, The (1990)', 'Rescuers Down Under, The (1990)', 'NeverEnding Story II: The Next Chapter, The (1990)', 'My Blue Heaven (1990)', 'Sheltering Sky, The (1990)', 'Edward Scis